### Задание 

Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/).   
Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество.   
Попробуйте добавить +1 рекуррентный в encoder и decoder.   
Попробуйте заменить GRU ячейки на lstm-ячейки.   
Оцените качество во всех случаях.   

Задание со звездочкой: Оптимизировать лекционный ноутбук. (добавить батчи и избавиться от прохода по Input тензору по одному за раз)

In [ ]:
# подгружаем данные в colab
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))    

In [ ]:
%matplotlib inline

In [ ]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 380911 sentence pairs
Trimmed to 22942 sentence pairs
Counting words...
Counted words:
rus 9108
eng 3945
['ты ужасна .', 'you re horrible .']


In [46]:
print(input_lang.name, output_lang.name, random.choice(pairs))

rus eng ['мы не уверены что сможем приити вечером .', 'we re not sure we can come tonight .']


The Encoder and The Decoder
-----------






In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
### Попробуйте добавить +1 рекуррентный в encoder и decoder.   

class ExtendedEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ExtendedEncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru_1 = nn.GRU(hidden_size, hidden_size)
        self.gru_2 = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden_1, hidden_2):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden_1 = self.gru_1(output, hidden_1)
        output, hidden_2 = self.gru_2(output, hidden_2)
        return output, hidden_1, hidden_2

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class ExtendedDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(ExtendedDecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru_1 = nn.GRU(hidden_size, hidden_size)
        self.gru_2 = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden_1, hidden_2):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden_1 = self.gru_1(output, hidden_1)
        output, hidden_2 = self.gru_2(output, hidden_2)
        output = self.softmax(self.out(output[0]))
        return output, hidden_1, hidden_2

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)        

In [ ]:
### Попробуйте заменить GRU ячейки на lstm-ячейки.   

class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        #lstm(input.view(len(input), self.batch_size, -1))
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)        

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    if type(encoder) == ExtendedEncoderRNN:
        encoder_hidden_1, encoder_hidden_2 = encoder.initHidden(), encoder.initHidden()
    elif type(encoder) == EncoderLSTM:
        encoder_hidden = ( encoder.initHidden(), encoder.initHidden() )
    else:
        encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        if type(encoder) == ExtendedEncoderRNN:
            encoder_output, encoder_hidden_1, encoder_hidden_2 = \
              encoder(input_tensor[ei], encoder_hidden_1, encoder_hidden_2)
        else:
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    if type(encoder) == ExtendedEncoderRNN:
        decoder_hidden_1, decoder_hidden_2 = encoder_hidden_1, encoder_hidden_2
    else:
        decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if type(encoder) == ExtendedEncoderRNN:
                decoder_output, decoder_hidden_1, decoder_hidden_2 = \
                  decoder(decoder_input, decoder_hidden_1, decoder_hidden_2)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):

            if type(encoder) == ExtendedEncoderRNN:
                decoder_output, decoder_hidden_1, decoder_hidden_2 = \
                  decoder(decoder_input, decoder_hidden_1, decoder_hidden_2)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        
        if type(encoder) == ExtendedEncoderRNN:
            encoder_hidden_1, encoder_hidden_2 = encoder.initHidden(), encoder.initHidden()  
        elif type(encoder) == EncoderLSTM:
            encoder_hidden = ( encoder.initHidden(), encoder.initHidden() )
        else:
            encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            if type(encoder) == ExtendedEncoderRNN:
                encoder_output, encoder_hidden_1, encoder_hidden_2 = \
                  encoder(input_tensor[ei], encoder_hidden_1, encoder_hidden_2)
            else:
                encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        if type(encoder) == ExtendedEncoderRNN:
            decoder_hidden_1 = encoder_hidden_1
            decoder_hidden_2 = encoder_hidden_2
        else:
            decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            if type(decoder) == ExtendedDecoderRNN:
                decoder_output, decoder_hidden_1, decoder_hidden_1 = \
                    decoder(decoder_input, decoder_hidden_1, decoder_hidden_2)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [63]:
hidden_size = 256
encoder_gru = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_gru = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder_gru, decoder_gru, 50000, print_every=5000)

0m 58s (- 8m 48s) (5000 10%) 3.1059
1m 55s (- 7m 43s) (10000 20%) 2.5746
2m 53s (- 6m 44s) (15000 30%) 2.3318
3m 51s (- 5m 47s) (20000 40%) 2.1149
4m 49s (- 4m 49s) (25000 50%) 1.9609
5m 47s (- 3m 51s) (30000 60%) 1.8141
6m 45s (- 2m 53s) (35000 70%) 1.7206
7m 43s (- 1m 55s) (40000 80%) 1.6125
8m 41s (- 0m 57s) (45000 90%) 1.5258
9m 40s (- 0m 0s) (50000 100%) 1.4442


In [64]:
evaluateRandomly(encoder_gru, decoder_gru)

> я о своем друге .
= i m talking about my friend .
< i m talking about the . . <EOS>

> ты гораздо моложе меня .
= you re way younger than i am .
< you re way younger than me am . <EOS>

> я готова сделать это для тебя .
= i m ready to do that for you .
< i m ready to do that for you . <EOS>

> он уезжает в нью иорк на следующеи неделе .
= he is leaving for new york next week .
< he is able to next week next week . <EOS>

> я нелегко схожусь с людьми .
= i m not good at meeting people .
< i m playing with the . . <EOS>

> я стараюсь остаться незамеченным .
= i m trying not to be seen .
< i m trying to be the . . <EOS>

> я слишком ленива для этого .
= i m too lazy to do that .
< i m too old for that . <EOS>

> я один .
= i m alone .
< i m alone . <EOS>

> я здесь только временно замещаю другого работника .
= i m just filling in here temporarily .
< i m just the only here . <EOS>

> я уверен что ты будешь доволен .
= i m sure that you ll be satisfied .
< i m sure you ll find with us . 

In [59]:
hidden_size = 256
encoder2 = ExtendedEncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder2 = ExtendedDecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder2, decoder2, 50000, print_every=5000)

1m 28s (- 13m 16s) (5000 10%) 3.1033
2m 57s (- 11m 50s) (10000 20%) 2.6527
4m 27s (- 10m 23s) (15000 30%) 2.3871
5m 56s (- 8m 55s) (20000 40%) 2.2077
7m 27s (- 7m 27s) (25000 50%) 2.0519
8m 57s (- 5m 58s) (30000 60%) 1.8801
10m 29s (- 4m 29s) (35000 70%) 1.7577
11m 59s (- 2m 59s) (40000 80%) 1.6346
13m 30s (- 1m 30s) (45000 90%) 1.5373
15m 1s (- 0m 0s) (50000 100%) 1.4735


In [60]:
evaluateRandomly(encoder2, decoder2)

> я рад что ты сегодня пришла .
= i m glad you came today .
< i i i i i i i i i i

> мне не хочется спать .
= i m not sleepy .
< i m not i m not i m not i

> мы точно не знаем где том .
= we re not sure where tom is .
< we re we re we re we re we re

> она купает ребенка .
= she is giving the baby a bath .
< she is she is she is she is she is

> я рад что дождь перестал .
= i m glad that the rain has stopped .
< i i i i i i i i i i

> вы взрослая .
= you re an adult .
< you re . you re . you re . you

> всю вторую половину дня она готовила .
= she spent all afternoon cooking .
< she is a we are she is a we are

> мне жаль что заставил вас ждать .
= i m sorry to have kept you waiting .
< i i m sorry i m sorry i m sorry

> я готов сделать заказ .
= i m ready to order .
< i m ready i am ready i am ready i

> они ее ищут .
= they re looking for it .
< they re looking they re looking they re looking they



In [61]:
hidden_size = 256
encoder_lstm = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder_lstm = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

trainIters(encoder_lstm, decoder_lstm, 50000, print_every=5000)

1m 1s (- 9m 9s) (5000 10%) 3.1818
2m 2s (- 8m 8s) (10000 20%) 2.7207
3m 3s (- 7m 8s) (15000 30%) 2.5153
4m 5s (- 6m 8s) (20000 40%) 2.3099
5m 8s (- 5m 8s) (25000 50%) 2.1601
6m 11s (- 4m 7s) (30000 60%) 2.0185
7m 12s (- 3m 5s) (35000 70%) 1.9154
8m 12s (- 2m 3s) (40000 80%) 1.8012
9m 13s (- 1m 1s) (45000 90%) 1.7221
10m 13s (- 0m 0s) (50000 100%) 1.6324


In [62]:
evaluateRandomly(encoder_lstm, decoder_lstm)

> я очень богат .
= i m very rich .
< i m very rich . <EOS>

> ты бледныи как полотно .
= you are as white as a sheet .
< you re like as a as . <EOS>

> вы очень грязная .
= you re really dirty .
< you re very generous . <EOS>

> он не друг а просто знакомыи .
= he is not a friend but an acquaintance .
< he is not but his his . . <EOS>

> я не интересуюсь футболом .
= i m not interested in football .
< i m not going to eat . <EOS>

> я добавляю последнии штрих .
= i m adding the last touch .
< i m the only one . . . <EOS>

> я уверена что том был бы в восторге .
= i m sure tom would be delighted .
< i m sure tom will be . . <EOS>

> я в восторге .
= i m fascinated .
< i m in the . . <EOS>

> ты не святая .
= you re no saint .
< you re not a . <EOS>

> я не буду тебе показывать .
= i m not going to show you .
< i m not going to tell you . <EOS>



Самым лучшим вариантом получился gru - и быстрее, и качество перевода тестовых примеров получилось интересным.  
Lstm не сильно отстает от gru и по качеству, по скорости, а визуально на тестовых примерах они ведут себя похоже.  
Схема из 2х GRU-модулей получилась наилучшей по формальным значениям loss функции, но визуально результат перевода получился довольно скудным - сетка часто повторяет первые 2-3 слова перевода.